# **Introduction to VectorDB**

## **What's Covered?**

1. What is a Vector Database?
2. Why Do We Need Vector Databases?
3. Applications
4. How are Vectors Stored?
5. What is indexing in VectorDBs?
6. Two Major Categories of Indexing
7. Flat (brute-force) index
8. Inverted File Index (IVF)
9. Hierarchical Navigable Small World (HNSW)
10. Major Vector Database Comparison

## **What is a Vector Database?**
A Vector Database (VectorDB) is a specialized type of database designed to store, manage, and search vector embeddings—high-dimensional representations of data that capture the semantic meaning of unstructured inputs like text, images, audio, or even code.

These embeddings are typically generated using machine learning models (e.g., transformers, sentence encoders, vision models), and they enable semantic similarity search—finding items that are "meaningfully" similar rather than just syntactically similar.

## **Why Do We Need Vector Databases?**
Traditional databases are optimized for exact matches or range queries on structured data (numbers, dates, text fields). However, with the rise of AI and Generative AI, we now deal heavily with unstructured data (e.g., user messages, support tickets, images), where meaning matters more than matching exact words or values.

Let’s consider this:
> You search “how to get my money back?”  
> A traditional search engine looks for documents with those exact words.  
> A semantic search engine powered by a VectorDB understands this is about refund policies, even if that phrase isn’t mentioned.

This leap from syntactic to semantic understanding is only possible with vector-based representations—and thus, vector databases.

## **Applications**
- **Semantic search engines -** Semantic search goes beyond keyword matching to understand the meaning of a query and return contextually relevant results. Example: Searching for “healthy snacks” returns “organic protein bars” even if the word "snack" isn't present.
- **Chatbot memory & knowledge retrieval -** Chatbots need to access relevant past conversations or knowledge to generate informed and contextual responses. Example: When a user asks, "What’s your refund policy?", the bot can fetch the relevant document snippet even if the question is phrased differently.
- **Recommendation systems -** Recommending content or products similar to what the user has interacted with. Example: Spotify uses vector similarity to recommend songs that match your recent listening pattern in mood and genre.
- **Content discovery -** Helping users explore new, relevant content that matches their interests or current context. Example: On a news platform, reading an article about climate change might surface other articles about global warming, carbon emissions, etc.
- **Document deduplication -** Identifying and removing duplicate or near-duplicate documents in large datasets. Example: Detecting that two job descriptions with slightly different formatting are actually the same.


| Application             | Role of VectorDB                                              |
| ----------------------- | ------------------------------------------------------------- |
| Semantic Search Engines | Finds contextually relevant results beyond keyword matches    |
| Chatbot Memory / RAG    | Retrieves most relevant knowledge chunks for better responses |
| Recommendation Systems  | Suggests similar items by comparing vectors                   |
| Content Discovery       | Enables intuitive exploration based on vector proximity       |
| Document Deduplication  | Identifies near-duplicate content using embedding similarity  |


## **How are Vectors Stored?**
Vectors are stored in specialized data structures optimized for similarity search:
- HNSW (Hierarchical Navigable Small World) - graph-based
index
- IVF (Inverted File) - clustering-based index
- Flat - brute force comparison (simple but slow)

Metadata is stored separately in a key-value store or document DB.  
IDs link the vector index with the metadata store

## **What is indexing in VectorDBs?**
Indexing is a fundamental technique used to organize vector data for efficient similarity search operations. In the context of vector databases, indexing creates specialized data structures that make nearest neighbor search operations faster and more efficient, especially when dealing with high-dimensional embeddings from text, images, audio, or other data types.

## **Two Major Categories of Indexing**
1. **Exact Indexing**
- Guarantees 100% accurate results
- Generally slower but provides perfect recall
- Example: Flat (brute-force) index
2. **Approximate Indexing (ANN - Approximate Nearest Neighbors)**
- Trades a small amount of accuracy for significant speed gains
- Uses various techniques to efficiently narrow down the search space
- Examples: IVF, HNSW, PQ, ANNOY

## **Flat (brute-force) index**

**What is the Flat Index?**
> A flat index is the simplest indexing method that performs an exhaustive search by comparing the query vector with every vector in the database.

**When to Use Flat Index?**
-  Small to medium datasets (typically ≤100K vectors)
-  When 100% accuracy is critical (e.g., security applications, financial matching)
-  During development and testing to establish baseline performance
-  When the dimensionality of vectors is relatively low

**How Flat Index Works?**
-  No preprocessing or clustering is performed
-  All vectors are stored in their original form
-  Each query requires direct distance computation with every stored vector
-  Typically uses Euclidean (L2) distance or cosine similarity

**Disadvantages**
- Search time grows linearly with dataset size (O(n) complexity)
- Memory intensive as all vectors must be stored in full precision
- Becomes impractical for large datasets

## **Inverted File Index (IVF)**


**What is IVF?**
IVF organizes vectors into clusters (using algorithms like k-means) and narrows search to only the most relevant clusters, dramatically speeding up search on large datasets.

**When to Use IVF?**
- Medium to large datasets (1M+ vectors)
- When you need a balance between speed and accuracy
- When you can accept approximate results (small accuracy trade-off)
- When you have a representative sample for training

**How IVF Works?**
1. Training Phase: Vectors are clustered into nlist clusters (centroids). nlist represents number of clusters.
2. Assignment Phase: Each vector is assigned to its nearest centroid
3. Search phase:
    - Query vector is compared to all centroids
    - Only vectors in the nprobe nearest clusters are examined
    - nprobe is a tunable parameter that balances speed vs accuracy. nprobe represents clusters to search.

**Distance Metrics**
- Uses the same distance metrics as flat indexes (typically L2 or cosine)
- The choice affects both clustering and search phases

**Tuning IVF Parameters**
- nlist (number of clusters): Higher values = faster search but lower accuracy
    - Rule of thumb: nlist = sqrt(N) where N is dataset size
- nprobe (clusters to search): Higher values = higher accuracy but slower search
    - Typical values: 1-10% of nlist

**Advantages**
- Much faster than flat index for large datasets
- Scalable to millions of vectors
- Adaptable trade-off between speed and accuracy via nprobe

**Disadvantages**
- Requires a training phase with representative data
- Not 100% accurate (may miss some relevant vectors)
- Performance depends on data distribution and clustering quality**

## **Hierarchical Navigable Small World (HNSW)**

**What is HNSW?**
Hierarchical Navigable Small World (HNSW) is a graph-based indexing method that creates a multi-layered graph structure for efficient navigation to nearest neighbors.

**When to Use HNSW?**
- Medium to large datasets where search speed is critical
- Real-time applications requiring low latency
- When high accuracy is important, but you can't use flat index
- When you have sufficient memory to store the graph structure

**How HNSW Works?**
1. Graph construction: Each vector becomes a node connected to its neighbors
2. Hierarchical structure: Multiple layers with varying connection densities
3. Search process:
    - Start at entry point in the top layer
    - Greedily move to closer neighbors
    - Descend through layers, refining the search
    - Stop at the bottom layer with the best candidates

**Distance Metrics in HNSW**
- Works with any distance metric (L2, cosine, etc.)
- Distance calculations are exact, but search path is approximate
- The choice of metric affects both graph construction and search

**Tuning HNSW Parameters**
- M (connections per node): Higher values = better accuracy but more memory
    - Typical values: 16-64
- efConstruction (build-time accuracy): Higher values = better index quality but slower build
    - Typical values: 100-500
- efSearch (search-time accuracy): Higher values = better search accuracy but slower search
    - Can be adjusted at query time

**Advantages**
- Excellent balance of speed and accuracy
- No training phase required
- Dynamic updates possible (can add vectors after construction)
- Works well with filtered queries

**Disadvantages**
- Memory intensive (stores both vectors and graph links)
- Index construction can be slow
- More complex implementation than other methods

## **Major Vector Database Comparison**

| Vector Database | Supported Index Types                    | Distance Metrics                    | Auto-Indexing? | Key Features                                                   |
|------------------|------------------------------------------|-------------------------------------|----------------|----------------------------------------------------------------|
| **FAISS**         | Flat, IVF, PQ, HNSW, SQ, LSH, combinations | L2, Cosine, Dot Product             | No             | Highly customizable, GPU support                              |
| **Pinecone**      | Proprietary (HNSW-based)                | Cosine, Dot Product, Euclidean      | Yes            | Fully managed, serverless, auto-scaling                        |
| **Weaviate**      | HNSW, Flat                              | Cosine                              | Yes            | Schema-based, multi-modal, GraphQL API                         |
| **Qdrant**        | HNSW, Scalar Quantization               | Cosine, Dot Product, Euclidean      | Yes            | Payload storage                                                |
| **Milvus**        | IVF, HNSW, PQ, ANNOY, combinations      | Euclidean, IP, Jaccard, others      | Yes            | Hybrid search, cloud/self-hosted                               |
| **Chroma**        | HNSW                                    | Cosine, L2, IP                      | Yes            | Simple API, embedding function integration                     |
| **Elasticsearch** | HNSW                                    | Cosine, Dot Product, L2             | Yes            | Text+vector search, mature ecosystem                           |
| **pgvector**      | IVF, HNSW                               | Cosine, L2, IP                      | Yes            | Postgres extension, familiar SQL interface                     |
